# Hull‑White 1F Calibration Notebook (Comparison of Market PV vs Model PV)

This notebook runs a calibration of a one‑factor Hull‑White model to co‑terminal ATM swaption volatilities
and compares market present values (PVs) to model PVs under three calibration settings.


## Importing libraries

In [ ]:
from QuantLib import *
import math
import sys
import statistics


## Market inputs

Provide the market inputs (swaption volatilities and zero rates) used for calibration.

In [ ]:
# User market data
market_vols_pct = [24.4, 24.08, 23.64, 23.3, 22.84]  # 1x5,2x4,3x3,4x2,5x1 (percent)
market_vols = [v / 100.0 for v in market_vols_pct]

# Zero rates (annual, in percent) at 1y..6y
zero_rates_pct = {1: 3.98, 2: 3.65, 3: 3.52, 4: 3.50, 5: 3.55, 6: 3.61}
zero_rates = {k: v / 100.0 for k, v in zero_rates_pct.items()}


## Dates, calendar and daycount


Set evaluation date, calendar and day count conventions; build a simple zero curve that extends beyond swaption maturities and enable extrapolation if necessary.

In [ ]:
tradeDate = Date(12, September, 2025)
Settings.instance().evaluationDate = tradeDate
settlementDate = tradeDate
calendar = TARGET()
dayCounter = Actual365Fixed()


## Build zero curve and enable extrapolation
Create co‑terminal swaption helpers (1x5, 2x4, 3x3, 4x2, 5x1) and compute market present values for each helper.

In [ ]:
# Build a simple zero curve and ensure it extends beyond required swaption maturities
dates = [settlementDate]
rates = []

for year in sorted(zero_rates.keys()):
    d = calendar.advance(settlementDate, Period(year, Years))
    dates.append(d)
    rates.append(zero_rates[year])

# Add extra long point (10y)
long_year = 10
long_date = calendar.advance(settlementDate, Period(long_year, Years))
dates.append(long_date)
rates.append(rates[-1])

zc = ZeroCurve(dates[1:], rates, dayCounter, calendar)
# enable extrapolation
try:
    zc.enableExtrapolation()
except Exception:
    pass

term_structure = YieldTermStructureHandle(zc)
try:
    term_structure.enableExtrapolation(True)
except Exception:
    pass


## Create co‑terminal swaption helpers and compute market PVs

Create co‑terminal swaption helpers (1x5, 2x4, 3x3, 4x2, 5x1) and compute market present values for each helper.

In [ ]:
helpers = []
n = len(market_vols)
index = USDLibor(Period(3, Months), term_structure)

for i in range(n):
    exercise_years = i + 1
    tenor_years = n - i
    vol = market_vols[i]
    helper = SwaptionHelper(
        Period(exercise_years, Years),
        Period(tenor_years, Years),
        QuoteHandle(SimpleQuote(vol)),
        index,
        Period(1, Years),    # fixed leg tenor = annual
        dayCounter,
        dayCounter,
        term_structure
    )
    helpers.append(helper)

# Save market PVs now (marketValue is independent of model engines)
market_pvs = []
for i, h in enumerate(helpers):
    try:
        mv = h.marketValue()
    except Exception as e:
        mv = None
        print(f"Warning: could not compute market PV for helper {i+1}: {e}")
    market_pvs.append(mv)


## Define calibration wrapper
Define a simple calibrator class that wraps the calibration routine and assigns the pricing engine to helpers.

In [ ]:
class ModelCalibrator:
    def __init__(self, endCriteria):
        self.endCriteria = endCriteria
        self.helpers = []

    def AddCalibrationHelper(self, helper):
        self.helpers.append(helper)

    def Calibrate(self, model, engine, curve, fixedParameters):
        # assign pricing engine to all calibration helpers
        for h in self.helpers:
            h.setPricingEngine(engine)
        method = LevenbergMarquardt()
        if len(fixedParameters) == 0:
            model.calibrate(self.helpers, method, self.endCriteria)
        else:
            model.calibrate(self.helpers, method, self.endCriteria,
                            NoConstraint(), [], fixedParameters)


## Helper to compute model PVs
Define a helper to compute model PVs for a given calibrated model.

In [ ]:
def compute_model_pvs_for_model(model):
    engine = JamshidianSwaptionEngine(model)
    model_pvs = []
    for h in helpers:
        # ensure pricing engine for modelValue()
        h.setPricingEngine(engine)
        try:
            pv = h.modelValue()
        except Exception as e:
            pv = None
            print(f"Error computing model PV for helper: {e}")
        model_pvs.append(pv)
    return model_pvs


## Calibrate and compare (three cases)
Run three calibration cases, compute model PVs for each and display a comparison table of market PV vs model PV and absolute differences.

In [ ]:
endCriteria = EndCriteria(1000, 100, 1e-8, 1e-8, 1e-8)
calibrator = ModelCalibrator(endCriteria)
for h in helpers:
    calibrator.AddCalibrationHelper(h)

def calibrate_and_compare(model, fixedParameters, label):
    # Use Jamshidian engine during calibration (as in sample)
    engine = JamshidianSwaptionEngine(model)
    try:
        calibrator.Calibrate(model, engine, term_structure, fixedParameters)
    except Exception as e:
        print(f"{label} calibration failed: {e}")
        return

    # compute model PVs after calibration
    model_pvs = compute_model_pvs_for_model(model)

    # print comparison table
    print("\n" + "="*70)
    print(f"{label}  --> calibrated params: a = {model.params()[0]:.8f}, sigma = {model.params()[1]:.8f}")
    print("-"*70)
    print("{:8s} {:10s} {:16s} {:16s} {:12s}".format("Swap", "MktVol", "MarketPV", "ModelPV", "AbsDiff"))
    print("-"*70)
    diffs = []
    for i in range(len(helpers)):
        swap_label = f"{i+1}x{n-i}"
        mvol = market_vols[i]
        mPV = market_pvs[i]
        moPV = model_pvs[i]
        if (mPV is None) or (moPV is None):
            diff = None
            diff_str = "N/A"
            mPV_str = "N/A" if mPV is None else f"{mPV:.8f}"
            moPV_str = "N/A" if moPV is None else f"{moPV:.8f}"
        else:
            diff = abs(moPV - mPV)
            diffs.append(diff)
            diff_str = f"{diff:.8e}"
            mPV_str = f"{mPV:.8f}"
            moPV_str = f"{moPV:.8f}"
        print("{:8s} {:10.4f} {:16s} {:16s} {:12s}".format(swap_label, mvol, mPV_str, moPV_str, diff_str))

    # RMS error
    if len(diffs) > 0:
        rms = math.sqrt(sum(d*d for d in diffs) / len(diffs))
        print("-"*70)
        print(f"RMS absolute PV error for {label}: {rms:.8e}")
    else:
        print("No valid diffs to compute RMS.")

# Case 1: calibrate both a and sigma
print("\nCase 1: calibrate both parameters (a and sigma)")
model_case1 = HullWhite(term_structure)   # default starting params
calibrate_and_compare(model_case1, [], "Case 1 (a and sigma free)")

# Case 2: fix mean reversion a = 0.05, calibrate sigma
print("\nCase 2: fix a=0.05, calibrate sigma")
model_case2 = HullWhite(term_structure, 0.05, 0.01)  # starting sigma 0.01
calibrate_and_compare(model_case2, [True, False], "Case 2 (fix a=0.05, sigma free)")

# Case 3: fix sigma = 0.01, calibrate a
print("\nCase 3: fix sigma=0.01, calibrate a")
model_case3 = HullWhite(term_structure, 0.05, 0.01)
calibrate_and_compare(model_case3, [False, True], "Case 3 (fix sigma=0.01, a free)")


## Summary of outputs

- Calibrated Hull‑White parameters (mean‑reversion and sigma) for each of three cases.
- A comparison table for each case showing: market volatility, market PV, model PV, and absolute PV difference for each swaption helper.
- RMS absolute PV error for each calibration case.
